## Cats and dogs classifier with hyperparameter tuning

Created this project after I failed miserably at classifying open or closed windows

I guess I do not have enough data? So I will try training the same setup with other kinds of images, 

like cats and dogs from kaggle

In [ ]:
!pip install keras-tuner
!pip install tensorboard
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
# libraries for displaying images
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import shutil
import math
from skimage import exposure
from tensorflow.keras.callbacks import CSVLogger
import pandas as pd
from sklearn.model_selection import train_test_split

import keras_tuner as kt
from keras_tuner import HyperParameters, BayesianOptimization

In [ ]:
# Perform all installations
!pip install tensorflow-gpu==2.5.0
!pip install tensorflow-datasets
!pip install tensorwatch
import tensorflow_datasets as tfds

# tfds makes a lot of progress bars and they take up a lot of screen space, so lets diable them
tfds.disable_progress_bar()

In [ ]:
WIDTH = 224
HEIGHT = 224
IMAGE_SIZE = (WIDTH, HEIGHT)
NUM_CLASSES = 2
CONTENT_DIR = r"..\contents"
TRAIN_DATA_DIR = os.path.join(CONTENT_DIR, r"catdog_imgs\train\train")
TEST_DATA_DIR = os.path.join(CONTENT_DIR, r"catdog_imgs\test\test")
LOG_DIR = r".\log"
LOG_HPARM = r".\log\hparm"
TRAIN_SAMPLES = 9
VAL_SAMPLES = 4
BATCH_SIZE = 10
EPOCH = 20
CLASS_THRESHOLD = 0.5
LEARNING_RATE = 0.001

from datetime import datetime
ts = datetime.now()
ts = f"{ts:%Y%m%d-%H%M%S}"

debug_mode = False

### Prepare the data
load image data from directory and preprocess it

In [ ]:
filenames=os.listdir(TRAIN_DATA_DIR)
categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)
df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})


In [ ]:
df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 TRAIN_DATA_DIR,x_col='filename',y_col='category',
                                                 target_size=IMAGE_SIZE,
                                                 class_mode='binary',
                                                 batch_size=batch_size)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    TRAIN_DATA_DIR, 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='binary',
    batch_size=batch_size
)

test_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)

test_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "./dogs-vs-cats/test/",x_col='filename',y_col='category',
                                                 target_size=IMAGE_SIZE,
                                                 class_mode='binary',
                                                 batch_size=batch_size)

Code to show the augmented images

In [ ]:
if debug_mode:
    for i in range(BATCH_SIZE):
        print(f'Run #{i}')
        img, label = train_generator.__next__()
        # classval = np.argmax(label[0], axis = 0)    # convert one-hot encoding to index
        # classkey = find_key(classval, iter_img.class_indices) # get class key from index value
        plt.imshow(img[0])
        plt.show()
        # print(f'img shape {img.shape}')
        # print(f'img label {classkey}\n')

    plt.show()

#### Load the pre-trained modfel and add in fine tuning layers
Now we define the model layers to freeze, discard to classification layers
then finally add in the fine tuning layers


In [ ]:

def model_maker(hp):
    # num_layers = hp.Int('num_layers', min_value=1, max_value=3)
    toFlatten = hp.Choice('toFlatten', values=[True, False])
    lr = hp.Choice('learning_rate', values=[1e-3, 1e-4])
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    # act = hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])

    custom_model = Sequential()
    # load model
    base_model = MobileNet(
        include_top=False,
        input_shape=(WIDTH, HEIGHT,3))
    
    for layer in base_model.layers[:]:
        layer.trainable = False

        
    custom_model.add(base_model)
    custom_model.add(GlobalAveragePooling2D())
    if toFlatten:
        custom_model.add(Flatten())

    # for num in range(num_layers):
    custom_model.add(Dense(hp.Int('units', min_value=16, max_value=128, step=16), activation = 'relu'))
    custom_model.add(Dropout(hp.Float('dropout', min_value=0.3, max_value=0.6, step=0.1)))

    custom_model.add(Dense(1, activation='sigmoid'))

    custom_model.compile(
        loss =  'binary_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])

    return custom_model


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
# from tensorflow.keras.optimizers import SGD,RMSprop,adam

def model_maker_scratch():
    from_scratch = Sequential()
    from_scratch.add(Convolution2D(224, 3,3,input_shape=(WIDTH, HEIGHT, 3)))
    from_scratch.add(Activation('relu'))
    from_scratch.add(Convolution2D(1000, 3, 3))
    from_scratch.add(Activation('relu'))
    from_scratch.add(MaxPooling2D(pool_size=(2, 2)))
    from_scratch.add(Dropout(0.5))
    # from_scratch.add(Convolution2D(64, 3, 3))
    from_scratch.add(Convolution2D(1000, 3, 3))
    from_scratch.add(Activation('relu'))
    from_scratch.add(MaxPooling2D(pool_size=(2, 2)))
    from_scratch.add(Dropout(0.5))
    # 

    from_scratch.add(Dense(256, activation='relu'))
    from_scratch.add(Dropout(0.5))
    from_scratch.add(Dense(1, activation='sigmoid'))
    # from_scratch.add(Dense(1))
    # from_scratch.add(Activation('sigmoid'))
    return(from_scratch)


In [ ]:
# switch between models
# model = model_maker_scratch()
# model = model_maker_inceptionResNetV2()
# model = model_maker()

# lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
# opt = tf.keras.optimizers.Adam(learning_rate=lr)

# model.compile(
#     loss = 'binary_crossentropy',
#     optimizer=opt,
#     metrics=['accuracy'])
# model.summary()


#Allow TensorBoard callbacks
tensorboard_callback = tf.keras.callbacks.TensorBoard(f"{LOG_DIR}\{ts}",
                                                      histogram_freq=1,
                                                      write_images=True)

csv_logger = CSVLogger('dogscats-training-' + 'log.csv',
                        append=True,
                        separator=';')



Hyperparameters configuration

In [ ]:
# tuner = kt.Hyperband(
# 	model,
# 	objective="val_accuracy",
# 	max_epochs=EPOCH,
# 	factor=3,
# 	seed=42,
# 	directory=LOG_DIR)

tuner = kt.BayesianOptimization(
	model_maker,
	objective="val_accuracy",
	max_trials=5,
	seed=42,
	directory=LOG_HPARM)

tuner.search(
	train_generator,
	validation_data=validation_generator,
	batch_size=BATCH_SIZE,
	epochs=3
)

In [ ]:
# hp = HyperParameters()
# hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
# hp.Int('units', min_value=32, max_value=512, step=32)

# tuner = BayesianOptimization(
#     hypermodel=model,  # Replace with your actual model
#     objective='val_loss',  # The metric to optimize (minimize)
#     max_trials=10,  # Number of trials (configurations) to test
#     num_initial_points=3,  # Initial random samples
#     alpha=0.0001,  # Noise level
#     beta=2.6  # Exploration-exploitation balance
# )

# tuner.search(train_data=train_generator, epochs=10, validation_data=val_generator)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best learning rate: {best_hps.get('learning_rate')}")
print(f"Best number of units: {best_hps.get('units')}")
best_model = tuner.hypermodel.build(best_hps)

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor = 'val_accuracy', min_delta=0.001, patience=10)

best_model.fit(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES)/BATCH_SIZE),
    epochs=EPOCH,
    callbacks=[tensorboard_callback,csv_logger,early_stopping],
    validation_data=val_generator,
    validation_steps=math.ceil(float(VAL_SAMPLES)/BATCH_SIZE)
    )


In [ ]:

# from gc import callbacks


# history = model.fit(
#     train_generator,
#     steps_per_epoch=math.ceil(float(TRAIN_SAMPLES)/BATCH_SIZE),
#     epochs=EPOCH,
#     callbacks=[tensorboard_callback,csv_logger,early_stopping],
#     validation_data=val_generator,
#     validation_steps=math.ceil(float(VAL_SAMPLES)/BATCH_SIZE))
    

In [ ]:

# params={
#         'batch_size':[64],  
#         'nb_epoch':[10, 15, 20], 
#         'units':[64, 128, 256], 
#         'dropout': [0.2, 0.3, 0.4],
#         } 
# gs=GridSearchCV(estimator=model, param_grid=params) 
# # now fit the dataset to the GridSearchCV object.  
# gs = gs.fit(train_generator)


# best_params=gs.best_params_ 
# accuracy=gs.best_score_ 

In [ ]:
# Start TensorBoard
%tensorboard --logdir ./log

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a 'history' object with training and validation loss
# Example: history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

# Retrieve training and validation loss
train_loss = best_model.history.history['loss'] 
val_loss = best_model.history.history['val_loss']

# Create a plot
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(train_loss) + 1), train_loss, label='Training Loss', marker='o')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss', marker='x')

# Customize the plot
plt.title('Training vs. Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()

In [ ]:
h5_path = f'..\contents\models\winclass_{ts}.h5'
best_model.save(h5_path)
tf.saved_model.save(best_model, "tmp/model/1/")

import shutil

# Specify the file you want to copy
source_file = h5_path
destination_file = 'model.h5'

# Copy the file
shutil.copyfile(source_file, destination_file)

print(f"File copied successfully from {source_file} to {destination_file}")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

model = load_model('model.h5')

In [ ]:
import cv2

# cv2.waitKey(0)
cam = cv2.VideoCapture(0)

result, cam_img = cam.read()
if result:
    
    # cv2.imshow("Captured image", cam_img)
    # equ_img = cv2.equalizeHist(cam_img)
    cv2.imwrite(TEST_DATA_DIR + r"\test_window.jpeg", cam_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
else:
    print("No image detected. Please try again.") 

from skimage import exposure, transform
from skimage.io import imread

img_path = r'../contents/windows_imgs/test/test_window.jpeg'
img = image.load_img(img_path, target_size=(224, 224))

# img = imread(img_path)
# img = exposure.equalize_hist(img)
# new_shape = (224, 224)
# img = transform.resize(img, new_shape, anti_aliasing=True)

img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(val_generator.class_indices)

# plt.figure(1,1)
plt.title(f"File Name: {img_path}, \n Prediction: Window is  {'Close' if prediction < CLASS_THRESHOLD else 'Open'}")
plt.imshow(img)
plt.show()    


In [ ]:
from skimage import exposure, transform
from skimage.io import imread

img_path = r'../contents/windows_imgs/test/test_window.jpeg'
img = image.load_img(img_path, target_size=(224, 224))

# img = imread(img_path)
# img = exposure.equalize_hist(img)
# new_shape = (224, 224)
# img = transform.resize(img, new_shape, anti_aliasing=True)

img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(val_generator.class_indices)

# plt.figure(1,1)
plt.title(f"File Name: {img_path}, \n Prediction: Window is  {'Close' if prediction < CLASS_THRESHOLD else 'Open'}")
plt.imshow(img)
plt.show()    


## List all images in test folder and make predictions

In [ ]:
import os
filelist = os.listdir(VAL_DATA_DIR + r"/close/")
# filelist = os.listdir(TEST_DATA_DIR)
jpegfiles = [file for file in filelist if file.endswith('.jpeg')]


Wanted to shrink the images down and display it inside a grid here.. work in progress

In [ ]:
# ## loop through file list and make a grid of images
# cols = 3
# rows = -(-jpegfiles.count // cols)
# fig, axes = plt.subplot(3, rows, figsize(18,6))
# axes = axes.flatten()

# # Plot each image in the corresponding subplot
# for i, image in enumerate(images):
#     img = image.load_img(os.path.join(VAL_DATA_DIR  + r"/close/",file), target_size=(WIDTH,HEIGHT))
#     # img = image.load_img(os.path.join(TEST_DATA_DIR,file), target_size=(WIDTH,HEIGHT))
#     img_array = image.img_to_array(img)
#     expanded_img_array = np.expand_dims(img_array, axis=0)
#     preprocessed_img = expanded_img_array / 255.  # Preprocess the image
#     prediction = model.predict(preprocessed_img)
#     print(prediction)
#     print(val_generator.class_indices)

#     ax = axes[i]
#     ax.imshow(img)
#     ax.title(f"File Name: {file}, \n Prediction: Window is  {'Close' if prediction < CLASS_THRESHOLD else 'Open'}")

#     ax.axis('off')  # Turn off axis labels

# plt.figure()

In [ ]:
for file in jpegfiles:
    img = image.load_img(os.path.join(VAL_DATA_DIR  + r"/close/",file), target_size=(WIDTH,HEIGHT))
    # img = image.load_img(os.path.join(TEST_DATA_DIR,file), target_size=(WIDTH,HEIGHT))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = expanded_img_array / 255.  # Preprocess the image
    prediction = model.predict(preprocessed_img)
    print(prediction)
    print(val_generator.class_indices)

    plt.title(f"File Name: {file}, \n Prediction: Window is  {'Close' if prediction < CLASS_THRESHOLD else 'Open'}")
    plt.imshow(img)
    plt.show()    


    